In [ ]:
#===== This code enable to produce total precipitation  from WRF-Chem and satallite observation ==========================
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset
import numpy as np
import cmaps
from matplotlib.patches import Patch
from pylab import *
from wrf import (to_np, getvar, interplevel, smooth2d, get_basemap, latlon_coords, ALL_TIMES)
from matplotlib.pyplot import figure
import matplotlib.gridspec as gridspec
import wrf
import scipy as sp
import scipy.ndimage
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import rcParams

"""Control font"""
matplotlib.rcParams['font.family'] = "sans-serif"
matplotlib.rcParams['font.sans-serif'] = "Times New Roman"

#=======Monsoon-WRF===
nc1 = Dataset("wrfout_d01_2017-05-01_00_00_00.nc")
nc2 = Dataset("wrfout_d01_2017-06-01_00_00_00")
nc3 = Dataset("wrfout_d01_2017-07-01_00_00_00")
nc4 = Dataset("wrfout_d01_2017-08-01_00_00_00")
nc5 = Dataset("wrfout_d01_2017-09-01_00_00_00")
tw1 = getvar(nc1, "T2",timeidx=-1) #,timeidx=ALL_TIMES,method="join", squeeze=False)     #ALL_TIMES)
tw2 = getvar(nc2, "T2",timeidx=-1) #timeidx=ALL_TIMES,method="join", squeeze=False)      #ALL_TIMES)
tw3 = getvar(nc3, "T2",timeidx=-1) #timeidx=ALL_TIMES,method="join", squeeze=False)     #ALL_TIMES)
tw4 = getvar(nc4, "T2",timeidx=-1) #timeidx=ALL_TIMES,method="join", squeeze=False)
tw5 = getvar(nc5, "T2",timeidx=-1) #timeidx=ALL_TIMES,method="join", squeeze=False)
tc1 = tw1-273
tc2 = tw2-273
tc3 = tw3-273
tc4 = tw4-273
tc5 = tw5-273
monsoon_wrf_tc = (tc1 + tc2 + tc3 + tc4 + tc5)/5
#===== Non-Monsoon-WRF=======
nc1 = Dataset("wrfout_d01_2017-03-01_00_00_00.nc")
nc2 = Dataset("wrfout_d01_2017-04-01_00_00_00.nc")
nc3 = Dataset("wrfout_d01_2017-01-01_00_00_00")
nc4 = Dataset("wrfout_d01_2017-02-01_00_00_00")
nc5 = Dataset("wrfout_d01_2017-10-01_00_00_00")
nc6 = Dataset("wrfout_d01_2017-11-01_00_00_00")
nc7 = Dataset("wrfout_d01_2017-12-01_00_00_00")
t1 = getvar(nc1, "T2",timeidx=-1)     #ALL_TIMES)
t2 = getvar(nc2, "T2",timeidx=-1)      #ALL_TIMES)
t3 = getvar(nc3, "T2",timeidx=-1)     #ALL_TIMES)
t4 = getvar(nc4, "T2",timeidx=-1)
t5 = getvar(nc5, "T2",timeidx=-1)
t6 = getvar(nc6, "T2",timeidx=-1)
t7 = getvar(nc7, "T2",timeidx=-1)
tc1 = t1-273
tc2 = t2-273
tc3 = t3-273
tc4 = t4-273
tc5 = t5-273
tc6 = t6-273
tc7 = t7-273
nonmonsoon_wrf_tc = (tc1 + tc2 + tc3 + tc4 + tc5 + tc6 + tc7)/7

#=== Monsoon-CRU==
cru_mon = "CRU_Monsoon_temp_new.nc" 
data1 = Dataset(cru_mon)
temp_mon1 = data1.variables['tmp'][:] 
monsoon_cru_tc = np.mean(temp_mon1[:,:,:], axis = 0)
lon1,lat1 = np.meshgrid(data1.variables['lon'][:], data1.variables['lat'][:])
#====Non-Monsoon-CRU====
cru_nonmon = "CRU_Non-Monsoon_temp_new.nc" 
data2 = Dataset(cru_nonmon)
temp_nonmon2 = data2.variables['tmp'][:]
nonmonsoon_cru_tc = np.mean(temp_nonmon2[:,:,:], axis = 0)
#=== Monsoon-ERA5===
era_mon = "ERA5-Monsoon_temp_new.nc" # the location of the data
data3 = Dataset(era_mon)
temp_mon3 = data3.variables['t2m'][:] # for t
temp_av_mon3 = np.mean(temp_mon3[:,:,:], axis = 0)
monsoon_era_tc = temp_av_mon3 - 273  # k to deg C
lon2,lat2 = np.meshgrid(data3.variables['longitude'][:], data3.variables['latitude'][:])
#=== Non-Monsoon-ERA5==
era_nonmon = "ERA5-Non-Monsoon_temp_new.nc" # the location of the data
data4 = Dataset(era_nonmon)
temp_nonmon4 = data4.variables['t2m'][:]
temp_av_nonmon4 = np.mean(temp_nonmon4[:,:,:], axis = 0)
nonmonsoon_era_tc = temp_av_nonmon4 - 273

#==== Now Plot======
fig,ax = plt.subplots(3,2,figsize=(8,8))

#===== WRF-Monsoon====
gridspec.GridSpec(3,2)
plt.subplot2grid((3,2), (0,0))

lats, lons = latlon_coords(monsoon_wrf_tc)
#m = Basemap(projection='merc',llcrnrlon=45,llcrnrlat=12,urcrnrlon=118,urcrnrlat=52,resolution='h')
m = Basemap(projection='lcc',width=12000000,height=9000000,area_thresh=1000,\
            lat_1=25,lat_2=25,lat_0=25,lon_0=90,llcrnrlon=45,llcrnrlat=0,\
             urcrnrlon=139,urcrnrlat=55,resolution='h')
#lon, lat = np.meshgrid(lons, lats)
x, y = m(to_np(lons), to_np(lats))
ax = m.pcolormesh(x,y,monsoon_wrf_tc,cmap=cmaps.BlueWhiteOrangeRed) #cmaps.WhiteBlueGreenYellowRed)
#========== Add Grid lines =======================================================================================
m.drawparallels(np.arange(10, 60, 15), linewidth=0.5, dashes=[4, 1], labels=[1,0,0,0], fontsize=10, color='black')
m.drawmeridians(np.arange(45, 125,15),linewidth=0.5, dashes=[4, 1],  color='black')
#========= Add countries boundasy =============================================================================
m.drawcountries()
m.drawcoastlines(linewidth=0.3)
plt.clim(-20,35)

plt.title('Monsoon [May-Sep]',fontsize=12)
plt.ylabel('WRF-Chem', labelpad=35,fontsize=12)

#========== WRF-Nonmonsoon ===============================
plt.subplot2grid((3,2), (0,1))
#m = Basemap(projection='merc',llcrnrlon=45,llcrnrlat=12,urcrnrlon=118,urcrnrlat=52,resolution='h')
m = Basemap(projection='lcc',width=12000000,height=9000000,area_thresh=1000,\
            lat_1=25,lat_2=25,lat_0=25,lon_0=90,llcrnrlon=45,llcrnrlat=0,\
             urcrnrlon=139,urcrnrlat=55,resolution='h')

##lon, lat = np.meshgrid(lons, lats)
x, y = m(to_np(lons), to_np(lats))
ax = m.pcolormesh(x,y,nonmonsoon_wrf_tc,cmap=cmaps.BlueWhiteOrangeRed) #cmaps.WhiteBlueGreenYellowRed)
m.drawparallels(np.arange(10, 60, 14), linewidth=0.5, dashes=[4, 1], color='black')
m.drawmeridians(np.arange(45, 125,15),linewidth=0.5, dashes=[4, 1],  color='black')
m.drawcountries()
m.drawcoastlines(linewidth=0.3)
plt.clim(-20,35)
plt.title('Non-monsoon [Oct-April]',fontsize=12)

#==== CRU- Monsoon ========
plt.subplot2grid((3,2), (1,0))
#m = Basemap(projection='merc',llcrnrlon=45,llcrnrlat=12,urcrnrlon=118,urcrnrlat=52,resolution='h')
m = Basemap(projection='lcc',width=12000000,height=9000000,area_thresh=1000,\
            lat_1=25,lat_2=25,lat_0=25,lon_0=90,llcrnrlon=45,llcrnrlat=0,\
             urcrnrlon=139,urcrnrlat=55,resolution='h')

x, y = m(lon1, lat1)
ax = m.pcolormesh(x,y,np.squeeze(monsoon_cru_tc),cmap=cmaps.BlueWhiteOrangeRed) 
m.drawparallels(np.arange(10, 60, 15), linewidth=0.5, dashes=[4, 1], labels=[1,0,0,0], fontsize=10, color='black')
m.drawmeridians(np.arange(45, 125,15),linewidth=0.5, dashes=[4, 1],  color='black')
m.drawcountries()
m.drawcoastlines(linewidth=0.3)
plt.clim(-20,35)
plt.ylabel('CRU', labelpad=35,fontsize=12)

#==== CRU- Non-Monsoon ========
plt.subplot2grid((3,2), (1,1))
#m = Basemap(projection='merc',llcrnrlon=45,llcrnrlat=12,urcrnrlon=118,urcrnrlat=52,resolution='h')
m = Basemap(projection='lcc',width=12000000,height=9000000,area_thresh=1000,\
            lat_1=25,lat_2=25,lat_0=25,lon_0=90,llcrnrlon=45,llcrnrlat=0,\
             urcrnrlon=139,urcrnrlat=55,resolution='h')

x, y = m(lon1, lat1)
ax = m.pcolormesh(x,y,np.squeeze(nonmonsoon_cru_tc),cmap=cmaps.BlueWhiteOrangeRed)
m.drawparallels(np.arange(10, 60, 15), linewidth=0.5, dashes=[4, 1], color='black')
m.drawmeridians(np.arange(45, 125,15),linewidth=0.5, dashes=[4, 1], color='black')
m.drawcountries()
m.drawcoastlines(linewidth=0.3)
plt.clim(-20,35)

#======== ERA5-Monsoon =============================
plt.subplot2grid((3,2), (2,0))
#m = Basemap(projection='merc',llcrnrlon=45,llcrnrlat=12,urcrnrlon=118,urcrnrlat=52,resolution='h')
m = Basemap(projection='lcc',width=12000000,height=9000000,area_thresh=1000,\
            lat_1=25,lat_2=25,lat_0=25,lon_0=90,llcrnrlon=45,llcrnrlat=0,\
             urcrnrlon=139,urcrnrlat=55,resolution='h')
x, y = m(lon2, lat2)
ax = m.pcolormesh(x,y,np.squeeze(monsoon_era_tc),cmap=cmaps.BlueWhiteOrangeRed)
m.drawparallels(np.arange(10, 60, 15), linewidth=0.5, dashes=[4, 1], labels=[1,0,0,0], fontsize=10, color='black')
m.drawmeridians(np.arange(45, 125,15),linewidth=0.5, dashes=[4, 1], labels=[0,0,0,1], fontsize=10, color='black')
m.drawcountries()
m.drawcoastlines(linewidth=0.3)
plt.clim(-20,35)
plt.ylabel('ERA5', labelpad=35,fontsize=12)
#======== ERA5-Non-Monsoon =============================
plt.subplot2grid((3,2), (2,1))
#m = Basemap(projection='merc',llcrnrlon=45,llcrnrlat=12,urcrnrlon=118,urcrnrlat=52,resolution='h')
m = Basemap(projection='lcc',width=12000000,height=9000000,area_thresh=1000,\
            lat_1=25,lat_2=25,lat_0=25,lon_0=90,llcrnrlon=45,llcrnrlat=0,\
             urcrnrlon=139,urcrnrlat=55,resolution='h')
x, y = m(lon2, lat2)
ax = m.pcolormesh(x,y,np.squeeze(nonmonsoon_era_tc),cmap=cmaps.BlueWhiteOrangeRed)
m.drawparallels(np.arange(10, 60, 15), linewidth=0.5, dashes=[4, 1], color='black')
m.drawmeridians(np.arange(45, 125,15),linewidth=0.5, dashes=[4, 1], labels=[0,0,0,1], fontsize=10, color='black')
m.drawcountries()
m.drawcoastlines(linewidth=0.3)
plt.clim(-20,35)

fig.subplots_adjust(top=0.967,
                    bottom=0.11,
                    left=0.085,
                    right=1,
                    hspace=0.025,
                    wspace=0.0)
# Colorbar
cax = fig.add_axes([0.11, 0.06,0.87,0.02])   # left, bottom, width, and height
cb = fig.colorbar(ax,  cax=cax, orientation='horizontal')
cb.set_label(label='Temperature [°C]',size=10)


plt.savefig("/mnt/g/2nd_Paper/Temperature.png",dpi=300)
#plt.show()


